In [1]:
#pip install ultralytics

  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.5.0
    Uninstalling typing-extensions-4.5.0:
      Successfully uninstalled typing-extensions-4.5.0



In [53]:
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
from IPython.core.magic import register_line_cell_magic
import yaml
from PIL import Image
import os
import seaborn as sns
from ultralytics import YOLO
from matplotlib.patches import Rectangle
import glob
import cv2

In [54]:
warnings.filterwarnings("ignore")

np.random.seed(12345)

%matplotlib inline

!wandb disabled

'wandb' is not recognized as an internal or external command,
operable program or batch file.


In [55]:
INPUT_DIR="data"

In [56]:
num_classes = 10
classes = [' Hardhat', ' Mask', ' NO-Hardhat', ' NO-Mask', ' NO-Safety Vest', ' Person', ' Safety Cone', ' Safety Vest', ' machinery', ' vehicle']

In [57]:
model=YOLO('best (1).pt')

WARNING ⚠️ best (1).pt appears to require 'dill', which is not in ultralytics requirements.
AutoInstall will run now for 'dill' but this feature will be removed in the future.
Recommend fixes are to train a new model using the latest 'ultralytics' package or to run a command with an official YOLOv8 model, i.e. 'yolo predict model=yolov8n.pt'
requirements: Ultralytics requirement ['dill'] not found, attempting AutoUpdate...

requirements: AutoUpdate success ✅ 4.4s, installed 1 package: ['dill']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



In [58]:
submission=pd.read_csv("SampleSubmission.csv")

In [59]:
submission

,ID,Hardhat,Mask,NO-Hardhat,NO-Mask,NO-Safety Vest,Person,Safety Cone,Safety Vest,machinery,vehicle
0,ID_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ID_2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ID_3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ID_4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ID_5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
191,ID_192,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
192,ID_193,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193,ID_194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
194,ID_195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
test_csv=pd.read_csv("Test.csv")

In [61]:
test_csv.columns

Index(['ID', 'filename'], dtype='object')

In [62]:
test_csv

,ID,filename
0,ID_1,006463_jpg.rf.02f19082420ecc5537b9d59abbe6050c...
1,ID_2,youtube-34_jpg.rf.03eacc444bae3c5caa3fef5c736c...
2,ID_3,IMG_0871_mp4-23_jpg.rf.03f872b1ed87ad7fadc85e0...
3,ID_4,youtube-596_jpg.rf.11a8a4ac01d8aadb80eeb0406df...
4,ID_5,ka_01181_png_jpg.rf.154ee4ef254eabd62e316be504...
...,...,...
191,ID_192,youtube-367_jpg.rf.fab8b016cb63c0928f40849db44...
192,ID_193,youtube-232_jpg.rf.f92f310106a11e01495ce8bfae0...
193,ID_194,construction-1027-_jpg.rf.e78f55761814a8726ea5...
194,ID_195,IMG_3100_mp4-3_jpg.rf.f05ef399880b1ac30e900df4...


In [65]:
submission['filename']=test_csv['filename']

In [66]:
for index, row in submission.iterrows():
    results = model.predict(source=os.path.join(INPUT_DIR, 'test', row['filename']), save=True)
    object_names = []
    for result in results:
        if result.boxes:
            box = result.boxes[0]
            class_id = int(box.cls)
            object_name = model.names[class_id]
            object_names.append(object_name)

    for class_name in object_names:
        submission.loc[index, class_name] = 1

submission.fillna(0, inplace=True)
print(submission.head())


image 1/1 C:\Users\Heni\Documents\studies\GODS 3.0\data\test\006463_jpg.rf.02f19082420ecc5537b9d59abbe6050c.jpg: 640x640 9 Hardhats, 8 NO-Masks, 9 NO-Safety Vests, 9 Persons, 1223.3ms
Speed: 10.0ms preprocess, 1223.3ms inference, 24.5ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict2

image 1/1 C:\Users\Heni\Documents\studies\GODS 3.0\data\test\youtube-34_jpg.rf.03eacc444bae3c5caa3fef5c736c3e40.jpg: 640x640 1 Hardhat, 1 NO-Safety Vest, 2 Persons, 1 machinery, 1 vehicle, 1104.0ms
Speed: 4.0ms preprocess, 1104.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict2

image 1/1 C:\Users\Heni\Documents\studies\GODS 3.0\data\test\IMG_0871_mp4-23_jpg.rf.03f872b1ed87ad7fadc85e09475ad37a.jpg: 640x640 1 Mask, 1 NO-Hardhat, 1 NO-Safety Vest, 1 Person, 1 vehicle, 1101.0ms
Speed: 2.0ms preprocess, 1101.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict2

i


image 1/1 C:\Users\Heni\Documents\studies\GODS 3.0\data\test\construction-192-_jpg.rf.56dd5c4c1396c6ad01be4fbec3bca46f.jpg: 640x640 1 NO-Safety Vest, 1 Person, 2 machinerys, 1121.5ms
Speed: 3.0ms preprocess, 1121.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict2

image 1/1 C:\Users\Heni\Documents\studies\GODS 3.0\data\test\IMG_3100_mp4-12_jpg.rf.58849ca6a931c2cfe1b17fb4206d9d82.jpg: 640x640 1 Mask, 1 NO-Hardhat, 1 NO-Safety Vest, 1 Person, 1169.1ms
Speed: 3.0ms preprocess, 1169.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict2

image 1/1 C:\Users\Heni\Documents\studies\GODS 3.0\data\test\youtube-510_jpg.rf.62f82e36d134d53587edb48f291986f4.jpg: 640x640 (no detections), 1191.1ms
Speed: 2.0ms preprocess, 1191.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict2

image 1/1 C:\Users\Heni\Documents\studies\GODS 3.0\data\test\

Results saved to runs\detect\predict2

image 1/1 C:\Users\Heni\Documents\studies\GODS 3.0\data\test\Movie-on-10-31-22-at-10_08-AM_mov-20_jpg.rf.aea4278b41550cdb75df5891a26052e7.jpg: 640x640 1 Mask, 1 NO-Hardhat, 1 NO-Safety Vest, 1 Person, 1113.2ms
Speed: 2.0ms preprocess, 1113.2ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict2

image 1/1 C:\Users\Heni\Documents\studies\GODS 3.0\data\test\youtube-738_jpg.rf.6f300b76e12b325f7c373c95ef319005.jpg: 640x640 1 Hardhat, 1 NO-Mask, 2 Persons, 1 Safety Vest, 1156.9ms
Speed: 2.0ms preprocess, 1156.9ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict2

image 1/1 C:\Users\Heni\Documents\studies\GODS 3.0\data\test\construction-1159-_jpg.rf.9f3172102a34edb157b556a73931c831.jpg: 640x640 1 machinery, 1155.1ms
Speed: 3.0ms preprocess, 1155.1ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict2


Speed: 3.0ms preprocess, 1257.9ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict2

image 1/1 C:\Users\Heni\Documents\studies\GODS 3.0\data\test\construction-2-_mp4-13_jpg.rf.cef0975976346515d438e0c2ce6c59db.jpg: 640x640 3 machinerys, 1242.4ms
Speed: 3.0ms preprocess, 1242.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict2

image 1/1 C:\Users\Heni\Documents\studies\GODS 3.0\data\test\NX_img_177_jpg.rf.c03709e5fadfe2109411f05a9e9bc25f.jpg: 640x640 2 Hardhats, 2 Masks, 1 NO-Hardhat, 3 NO-Safety Vests, 4 Persons, 1192.7ms
Speed: 3.0ms preprocess, 1192.7ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict2

image 1/1 C:\Users\Heni\Documents\studies\GODS 3.0\data\test\youtube-198_jpg.rf.e89faeb9765c6bd6cece5434d140f4af.jpg: 640x640 1 Hardhat, 1 Person, 1 Safety Vest, 1 machinery, 1221.9ms
Speed: 3.0ms preprocess, 1221.9ms inference

Speed: 3.0ms preprocess, 1201.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict2

image 1/1 C:\Users\Heni\Documents\studies\GODS 3.0\data\test\youtube-399_jpg.rf.3472ef7c01bb9092da48f0d562581121.jpg: 640x640 5 machinerys, 2 vehicles, 1160.4ms
Speed: 2.0ms preprocess, 1160.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict2

image 1/1 C:\Users\Heni\Documents\studies\GODS 3.0\data\test\02646_jpg.rf.5c93ba95bdc03808bcf872c7218ac5ef.jpg: 640x640 1 Hardhat, 1 NO-Mask, 1 NO-Safety Vest, 1 Person, 1180.5ms
Speed: 3.0ms preprocess, 1180.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict2

image 1/1 C:\Users\Heni\Documents\studies\GODS 3.0\data\test\IMG_3103_mp4-18_jpg.rf.39e564f01ba6048fc34853e99f3de11b.jpg: 640x640 1 Mask, 1 NO-Hardhat, 1 NO-Safety Vest, 1 Person, 1175.0ms
Speed: 2.0ms preprocess, 1175.0ms inference, 1.0ms postpr

Results saved to runs\detect\predict2

image 1/1 C:\Users\Heni\Documents\studies\GODS 3.0\data\test\youtube-194_jpg.rf.601da70a10e87c30c693ffbee53b17a7.jpg: 640x640 1 Person, 1 Safety Vest, 1 machinery, 1195.1ms
Speed: 3.0ms preprocess, 1195.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict2

image 1/1 C:\Users\Heni\Documents\studies\GODS 3.0\data\test\YouTube_FreeStockFootage_People-wearing-face-mask_Empty-Street_Covid19-D_DbgrvhlGs-720p_mp4-66_jpg.rf.72af13fb1f20520e0d41547037ced88b.jpg: 640x640 5 NO-Hardhats, 6 NO-Safety Vests, 11 Persons, 1172.2ms
Speed: 2.0ms preprocess, 1172.2ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict2

image 1/1 C:\Users\Heni\Documents\studies\GODS 3.0\data\test\IMG_9949-1-_mp4-20_jpg.rf.6ff2adfaf91058088d86084475d4d575.jpg: 640x640 1 Mask, 1 NO-Hardhat, 1 NO-Safety Vest, 1 Person, 1229.4ms
Speed: 3.0ms preprocess, 1229.4ms inference, 1.0ms postp

Results saved to runs\detect\predict2

image 1/1 C:\Users\Heni\Documents\studies\GODS 3.0\data\test\Movie-on-10-31-22-at-10_08-AM_mov-22_jpg.rf.a02b881bb190fa4a30026a648d13d278.jpg: 640x640 1 Mask, 1 NO-Hardhat, 1 NO-Safety Vest, 1 Person, 1165.1ms
Speed: 3.0ms preprocess, 1165.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict2

image 1/1 C:\Users\Heni\Documents\studies\GODS 3.0\data\test\class1_267_jpg.rf.ab7a08d97aba5b0748e976df6e65700a.jpg: 640x640 1 Hardhat, 1 NO-Hardhat, 3 NO-Masks, 2 NO-Safety Vests, 2 Persons, 1175.1ms
Speed: 2.0ms preprocess, 1175.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict2

image 1/1 C:\Users\Heni\Documents\studies\GODS 3.0\data\test\4_jpg.rf.a937c41e1fc4d92e9e1f8c2d9efeb87d.jpg: 640x640 1 Hardhat, 3 Persons, 4 Safety Vests, 1 machinery, 1121.2ms
Speed: 2.0ms preprocess, 1121.2ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

Results saved to runs\detect\predict2

image 1/1 C:\Users\Heni\Documents\studies\GODS 3.0\data\test\construction-4-_mp4-20_jpg.rf.cb627b855fa08d83357febb83f6ad4bc.jpg: 640x640 2 machinerys, 1127.6ms
Speed: 2.0ms preprocess, 1127.6ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict2

image 1/1 C:\Users\Heni\Documents\studies\GODS 3.0\data\test\youtube-342_jpg.rf.d7e55a17800f8d87313d7b6f33256ea9.jpg: 640x640 1 NO-Safety Vest, 3 Persons, 2 Safety Vests, 3 machinerys, 6 vehicles, 1134.9ms
Speed: 3.0ms preprocess, 1134.9ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict2

image 1/1 C:\Users\Heni\Documents\studies\GODS 3.0\data\test\youtube-226_jpg.rf.dd9d045ca17155289bcebe722e8a86e9.jpg: 640x640 2 machinerys, 1198.4ms
Speed: 3.0ms preprocess, 1198.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict2

image 1/1 C:\Users\Heni\Document

In [67]:
# Define the columns you want to keep
columns_to_keep = ['ID', ' Hardhat', ' Mask', ' NO-Hardhat', ' NO-Mask', ' NO-Safety Vest', ' Person', ' Safety Cone', ' Safety Vest', ' machinery', ' vehicle']

# Create a new DataFrame with only the specified columns
extracted_submission = submission[columns_to_keep]

# Write the DataFrame to a CSV file
extracted_submission.to_csv('SubmissionBest1.csv', index=False)
